<a href="https://colab.research.google.com/github/IGTarciano/ScriptRelatorioAutomatizado/blob/main/Scrip_relatorio_de_compras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # # Criador: Igor T Gomes;
# # # Data: ultima atualização: 18/05/2025;
# # #

# 📦 Instalação de dependências (se necessário) / Scrip: relatorio - resumo da emergencia;
!pip install openpyxl --quiet
# Importando bibliotecas:
import pandas as pd
import numpy as np

# 📁 1. Upload do arquivo de dados

try:
      from google.colab import files
      uploaded = files.upload()
      if not uploaded:
          raise ValueError("Nenhum arquivo foi carregado.") # Trantando erro do arquivo

      # Leitura do arquivo:
      Data_base = list(uploaded.keys())[0]
      Base_resumo_emergencia = pd.read_excel(Data_base)
except IndexError as e:
  print('Erro - Index.Erro',e)
except Exception as e:
  print('Erro - Exception.Erro',e)


try: # Processando relatorio e suas dependencias
      # Valor total do item por pedido;
      Base_resumo_emergencia['Valor_Total'] = Base_resumo_emergencia['PedidoCompraItem_Qtde'] * Base_resumo_emergencia['PedidoCompraItem_ValorTotal']
      # Definindo fornecedor:
      Filtro_fornecedor = Base_resumo_emergencia[Base_resumo_emergencia['Pessoa_Nome'] == 'TOYOTA DO BRASIL LTDA'].copy()
      ###
      # Selecionando colunas desejadas;
      colunas_p_analise = [
          'PedidoCompra_EmpresaCod',
          'PedidoCompra_Codigo',
          'PedidoCompra_NroMontadora',
          'Pessoa_Nome',
          'PedidoCompra_DataCriacao',
          'ProdutoMarca_Referencia',
          'Produto_Descricao',
          'PedidoCompraItem_Qtde',
          'PedidoCompraItem_QtdeAtendida',
          'Saldo',
          'PedidoCompraItem_ValorTotal',
          'Valor_Total',
          'TipoPedido_Descricao',
          'ProdutoEstoque_ClasABCLetraPopularidade'
          ]
      cols_existentes_analitico = [col for col in colunas_p_analise if col in Filtro_fornecedor.columns if col in Filtro_fornecedor.columns]
      resumo_emergencia = Filtro_fornecedor[cols_existentes_analitico].copy()

      # Renomeando as columns:

      rename = {
          'PedidoCompra_EmpresaCod':'Filial',
          'PedidoCompra_Codigo':'Cod. Interno',
          'PedidoCompra_NroMontadora':'Cod. Montadora',
          'Pessoa_Nome':'Fornecedor',
          'PedidoCompra_DataCriacao':'Data Pedido',
          'ProdutoMarca_Referencia':'Referencia',
          'Produto_Descricao':'Descrição',
          'PedidoCompraItem_Qtde':'Qtde Pedida',
          'PedidoCompraItem_QtdeAtendida':'Qtde Atendida',
          'Saldo':'Saldo a ser atendido',
          'PedidoCompraItem_ValorTotal':'Valor unitario do item',
          'Valor_Total':'Valor total do item',
          'TipoPedido_Descricao':'Tipo de pedido',
          'ProdutoEstoque_ClasABCLetraPopularidade':'Classificação ABC'
      }

      # # #
      resumo_emergencia_analitico = resumo_emergencia.rename(columns=rename)

      ### Alterando o nome das filiais:

      ### Dicionario para mudar o nome das filiais:
      mapeamento_filiais = {
          180:'IMBIRIBEIRA',
          182:'CARUARU',
          186:'NATAL',
          187:'RUI BARBOSA',
          188:'SÃO LUIS',
          189:'ARACAJU',
          190:'MANAUS',
          191:'TIRIRICAL',
          192:'PETROLINA',
          193:'GARANHUNS',
          194:'BOA VISTA',
          195:'MOSSORO',
          196:'OLINDA',
          197:'AFOGADOS',
          198:'CARPINA',
          199:'CAICO',
          200:'LEXUS',
          201:'SERRA'
          }
      resumo_emergencia_analitico['Filial'] = resumo_emergencia_analitico['Filial'].map(mapeamento_filiais)

      ### Filtrando os tipos de pedido:
      # resumo_emergencia_analitico = resumo_emergencia_analitico[resumo_emergencia_analitico['Tipo de pedido'] != 'R - REPOSICAO - TDB']
      resumo_emergencia_analitico = resumo_emergencia_analitico[resumo_emergencia_analitico['Tipo de pedido'] != 'R - REPOSICAO - TDB']

      # # # Tabela de itens D, E e inativos;

      categoria_class = ['D1','D2','E0','E1','F','G','H','I','J']
      # # # Colocar descrição do item # # #
      if 'Classificação ABC' in resumo_emergencia_analitico.columns:
          # friltrando os itens D,E e Inativos
          df_itens_D_E_Inativos = resumo_emergencia_analitico[resumo_emergencia_analitico['Classificação ABC'].isin(categoria_class)]
          Resumo_emergencia_D_E_Inativos = df_itens_D_E_Inativos.groupby(['Filial','Descrição','Referencia','Classificação ABC','Valor total do item'])['Qtde Pedida'].sum().reset_index() # .reset_index() transforma as chaves do groupby em colunas novamente
      else:
          print("Erro: A coluna 'ProdutoEstoque_ClasABCLetraPopularidade' não foi encontrada. O resumo de emergência não pôde ser gerado.")

      # # # Tabela de itens A,B e C;
      # # # Colocar descrição do item # # #

      categoria_class = ['A1','A2','A3','B1','B2','B3','C1','C2','C3']

      if 'Classificação ABC' in resumo_emergencia_analitico.columns:
          # friltrando os itens A,B e C
          df_itens_A_B_C = resumo_emergencia_analitico[resumo_emergencia_analitico['Classificação ABC'].isin(categoria_class)]
          Resumo_emergencia_A_B_C = df_itens_A_B_C.groupby(['Filial','Descrição','Referencia','Classificação ABC','Valor total do item'])['Qtde Pedida'].sum().reset_index() # A utilização de df['coluna'].isin(lista) é a forma correta e eficiente de filtrar por múltiplos valores.
      else:
          print("Erro: A coluna 'ProdutoEstoque_ClasABCLetraPopularidade' não foi encontrada. O resumo de emergência não pôde ser gerado.")

      ### Filtrando os tiposde pedido:
      # resumo_emergencia_analitico = resumo_emergencia_analitico[resumo_emergencia_analitico['Tipo de pedido'] != 'R - REPOSICAO - TDB']
      resumo_emergencia_analitico = resumo_emergencia_analitico[resumo_emergencia_analitico['Tipo de pedido'] != 'R - REPOSICAO - TDB']

      ### Tratando valores NaN;
      mensagem_class = 'S/class' ### seções vazias na classificação serão substituidas por S/class;
      resumo_emergencia_analitico['Classificação ABC'] = resumo_emergencia_analitico['Classificação ABC'].fillna(mensagem_class)
      resumo_emergencia_analitico.dropna(subset=['Cod. Montadora'],inplace=True) ### Removendo linhas onde o numero da montadora está vazio;

      # Criando o relatorio final:
      try:
          from google.colab import drive
          drive.mount('/content/drive')
          caminho_arquivo = '/content/drive/MyDrive/Acompanhamento/Resumo_emergencia.xlsx'
          with pd.ExcelWriter(caminho_arquivo, engine='openpyxl') as writer:
              resumo_emergencia_analitico.to_excel(writer, sheet_name='Resumo_emergencia_Analitico')
              Resumo_emergencia_A_B_C.to_excel(writer, sheet_name='Resumo_emergencia_A_B_C')
              Resumo_emergencia_D_E_Inativos.to_excel(writer, sheet_name='Resumo_emergencia_D_E_Inativos')

              print(f"RELATORIO SALVO COM SUCESSO: {caminho_arquivo}")

      except Exception as e:
        print('Erro - Exception.Erro',e)


      # # #:
except IndexError as e:
  print('Erro - Index.Erro',e)
except KeyError as e:
  print('Erro - Key.Erro',e)
except TypeError as e:
  print('Erro - Type.Erro',e)
except Exception as e:
  print('Erro - Exception.Erro',e)